# Import des bibliothèques

In [81]:
from bs4 import BeautifulSoup
import pandas as pd
from urllib import request
import re
import requests
from datetime import datetime
import locale
import pandas as pd
from geopy.geocoders import Nominatim
import folium
import time
import trainline
import io

# Get DF

In [47]:
url = "https://www.billetreduc.com/R/4/liste.htm"
response = request.urlopen(url).read()

soup = BeautifulSoup(response, 'html')

In [48]:
data = []
page_number = soup.find('td', string=re.compile(r'pages de résultats', re.IGNORECASE)).get_text()
total_pages = int(re.search(r'\d+', page_number).group())

lieu_url = []

for page_number in range(1, total_pages + 1):
    page_url = f"{url}?LISTEPEpg={page_number}"
    response = requests.get(page_url)
    soup = BeautifulSoup(response.content, 'html.parser')

    preliste_table = soup.find('table', id='preliste')
    # Extract information from each row in the table
    for event in preliste_table.find_all('tr', class_=''):
        title_element = event.find('h4')
        title = title_element.text.strip() if title_element else None

        description_element = event.find('div', class_='libellepreliste')
        description = description_element.text.strip() if description_element else None

        date_element = event.find('span', class_='lieu')
        if date_element and 'Prochaines dates:' in date_element.text:
            date = date_element.text.split('Prochaines dates:')[1].strip()
        else:
            date = None

        venue_element = event.find('span', class_='lieu')
        if venue_element:
            lieu_element = venue_element.find('a').get('href')
            lieu_element ='https://www.billetreduc.com' + lieu_element

            lieu_url.append(lieu_element)
        #     try:
        #         responselieu = requests.get(lieu_element)
        #         if responselieu.status_code != 200:
        #             print(f"Failed to retrieve the webpage: {lieu_element}")
        #     except Exception as e:
        #         print(f"Error occurred while fetching the webpage: {lieu_element}, Error: {e}")
        #     soup_lieu = BeautifulSoup(responselieu.content, 'html.parser')

            
        #     if soup_lieu.find('td', {'class': 'bgbeige'}).find('h5'):
        #         venue = soup_lieu.find('td', {'class': 'bgbeige'}).find('h5').text
        #         print(venue)
        #     else:
        #         venue=None
        # else:
        #     venue = None
        # venue = venue_element.text.split(',')[0].strip() if venue_element else None 
        # venue = venue + ", " if venue_element else None
        # venue = venue+venue_element.text.split(',')[1].strip().split('Prochaines dates')[0].strip() if venue_element else None

        price_element = event.find('span', class_='prixli')
        price = price_element.text.strip() if price_element else None

        # Handle the case where 'div' with class 'reduc' is not found
        discount_element = event.find('div', class_='reduc')
        discount = discount_element.text.strip() if discount_element else None

        data.append({
            "titre": title,
            "description": description,
            "date": date,
            "lieu": lieu_element,
            "prix": price,
            "reduction": discount
        })

df = pd.DataFrame(data)

In [49]:
df = df.dropna(subset="titre").reset_index(drop=True)

In [50]:
lieu_url = list(set(lieu_url))
results_lieu = {}

for url in lieu_url:
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')

    td_elements = soup.find_all('td', {'class': 'bgbeige'})

    for td in td_elements:
        h5_element = td.find('h5')
        if h5_element:
            # Ajouter le texte de l'élément h5 au dictionnaire
            results_lieu[url] = h5_element.text.strip()

In [51]:
df["lieu"] = df["lieu"].map(results_lieu)
df['lieu'] = df['lieu'].str.replace(r'\s+,', ',', regex=True)

Après avoir récupéré le csv, pas besoin de lancer la cellule ci-dessous

In [52]:
df["lieu"].to_csv("lieu_concert.csv", index=False)

In [99]:
df.to_csv("concerts.csv", index=False)

# Convert Date

In [54]:
def split_date(date_string):
    if pd.notna(date_string):
        if "au" in date_string:
            start_date, end_date = map(str.strip, date_string.split("au"))
        else:
            start_date, end_date = date_string, None

        # Remove "Le " or "Du " from the start of the dates
        start_date = start_date.replace("Le ", "").replace("Du ", "").strip()
    else:
        start_date, end_date = None, None

    return start_date, end_date

In [55]:
df[['start_date', 'end_date']] = df['date'].apply(split_date).apply(pd.Series)

In [56]:
locale.setlocale(locale.LC_TIME, 'fr_FR')
df["start_date"] = df["start_date"].apply(lambda x: datetime.strptime(x, "%A %d %B %Y") if x else None)
df["end_date"] = df["end_date"].apply(lambda x: datetime.strptime(x, "%A %d %B %Y")if x else None)

# Geocodage

In [57]:
dflieu = pd.read_csv("lieu_concert.geocoded.csv")
dflieu = dflieu[['lieu', 'result_city', 'latitude', 'longitude']]
dflieu = dflieu.drop_duplicates(subset=['lieu'])

In [58]:
df = pd.merge(df, dflieu, on='lieu', how='inner')

In [60]:
latitude_moyenne = df['latitude'].mean()
longitude_moyenne = df['longitude'].mean()
carte = folium.Map(location=[latitude_moyenne, longitude_moyenne], zoom_start=7)

for idx, row in df.iterrows():
    folium.Marker(
        location=[row['latitude'], row['longitude']],
        popup=row['titre'],
        icon=folium.Icon(icon="cloud"),
    ).add_to(carte)

In [61]:
carte

# Trainline

Liste des villes où les évènements ont lieu le plus souvent selon le site scrappé (billetreduc.)

In [35]:
grd_villes = ['Paris', 'Avignon', 'Bordeaux', 'Lille', 'Lyon', 'Marseille',
              'Montpellier', 'Nantes', 'Nice', 'Toulouse']

Nous décidons de garder uniquement les évènements qui se situent dans ces grandes villes.

In [103]:
df.sort_values("end_date", ascending=False)

,titre,description,date,lieu,prix,reduction,start_date,end_date,result_city,latitude,longitude
13,Gospel Dream,Créée en 1990 par son chef de choeur michel m'...,Du samedi 27 janvier 2024 au mardi 31 décembre...,"23 avenue George V, 75008 Paris","27,00€",jusqu'à -16%,2024-01-27,2024-12-31,Paris,48.867508,2.300939
83,Concerto Pour Une Nuit De Noël De Corelli + Le...,Sa formation à géométrie variable permet à l'o...,Du vendredi 20 décembre 2024 au vendredi 27 dé...,"Place Saint Germain des Prés, 75006 Paris","22,00€",None,2024-12-20,2024-12-27,Paris,48.854182,2.333399
588,Montmartre Enchanté Noël : Visite Contée Et Ch...,Et si vous retrouviez l'esprit de noel le temp...,Du samedi 7 décembre 2024 au mercredi 25 décem...,"Métro Abbesses, 75018 Paris","8,95€",jusqu'à -30%,2024-12-07,2024-12-25,Paris,48.884943,2.337073
79,"Les 4 Saisons De Vivaldi, Ave Maria Et Célèbre...",Par l'orchestre hélios et glen rouxel en violo...,Du samedi 3 février 2024 au samedi 14 décembre...,"Place Saint Germain des Prés, 75006 Paris","22,00€",None,2024-02-03,2024-12-14,Paris,48.854182,2.333399
2,Gospel Dream,Créée en 1990 par son chef de choeur michel m'...,Du vendredi 3 mai 2024 au vendredi 13 décembre...,"Place de la Madeleine, 75008 Paris","27,00€",jusqu'à -16%,2024-05-03,2024-12-13,Paris,48.870062,2.324455
...,...,...,...,...,...,...,...,...,...,...,...
584,Hommage À John Williams : Symphonie Des Étoiles,Hommage à john williams,Le dimanche 9 juin 2024,"50, quai Charles de Gaulle, 69006 Lyon","39,90€",None,2024-06-09,NaT,Lyon,45.785300,4.854545
585,Kebu,"Venez découvrir kebu, compositeur et artiste f...",Le mardi 18 juin 2024,"61 rue du Château d’Eau, 75010 Paris","27,00€",jusqu'à -16%,2024-06-18,NaT,Paris,48.872495,2.355162
587,Humain Cool,"Humain cool auteur, compositeur, interprète, h...",Le samedi 23 novembre 2024,"1 Place Antonin Jutard, 69003 Lyon","25,00€",None,2024-11-23,NaT,Lyon,45.756531,4.841261
589,Divine Récital Noël Chansons Et Chants Sacrés,Récital de noel en chansons et chants sacrés à...,Le dimanche 8 décembre 2024,"11 rue Yvonne Le Tac, 75018 Paris","5,95€",jusqu'à -59%,2024-12-08,NaT,Paris,48.884063,2.340330


In [96]:
df = df[df['result_city'].isin(grd_villes)]

In [97]:
df.reset_index(drop=True, inplace=True)

In [ ]:
all_results = pd.DataFrame()

for departure in grd_villes:
    for arrival in grd_villes:
        if departure != arrival:
            resultats = trainline.search(
                departure_station=departure,
                arrival_station=arrival,
                from_date="25/01/2024 00:01",
                to_date="25/01/2024 23:59")

            data_io = io.StringIO(resultats.csv())
            traindata = pd.read_csv(data_io, sep=";")
            
            # Ajoutez les colonnes departure_station et arrival_station
            traindata = traindata.assign(departure_station=departure, arrival_station=arrival)
            
            # Ajoutez les résultats au DataFrame global
            all_results = pd.concat([all_results, traindata])

# Affichez les résultats
print(all_results)

In [71]:
resultats = trainline.search(
	departure_station="Toulouse",
	arrival_station="Bordeaux",
	from_date="25/01/2024 08:00",
	to_date="25/01/2024 21:00")

In [83]:
data_io = io.StringIO(resultats.csv())

In [87]:
resultats

In [84]:
traindata = pd.read_csv(data_io, sep=";")

,departure_date,arrival_date,duration,number_of_segments,price,currency,transportation_mean,bicycle_reservation
0,25/01/2024 08:19,25/01/2024 10:40,02h21,1,"22,0",EUR,train,"10,0"
1,25/01/2024 08:19,25/01/2024 10:40,02h21,1,"69,0",EUR,train,unavailable
2,25/01/2024 10:05,25/01/2024 13:10,03h05,1,"5,99",EUR,coach,unavailable
3,25/01/2024 10:31,25/01/2024 12:40,02h09,1,"22,0",EUR,train,"10,0"
4,25/01/2024 11:23,25/01/2024 13:38,02h15,1,"65,0",EUR,train,unavailable
5,25/01/2024 11:23,25/01/2024 13:38,02h15,1,"17,5",EUR,train,"10,0"
6,25/01/2024 11:23,25/01/2024 13:38,02h15,1,"23,5",EUR,train,unavailable
7,25/01/2024 11:40,25/01/2024 14:40,03h00,1,"7,96",EUR,coach,unavailable
8,25/01/2024 12:27,25/01/2024 14:40,02h13,1,"22,0",EUR,train,"10,0"
9,25/01/2024 12:27,25/01/2024 14:40,02h13,1,"24,0",EUR,train,unavailable
